In [15]:
import pandas as pd
import sklearn

<h1> Configurando DataSet

In [16]:
file = pd.read_csv('Consumo_cerveja.csv')
file['Data'] = pd.to_datetime(file['Data'])
file["Dia da Semana"] = file["Data"].dt.weekday

file['Temperatura Media (C)'] = file['Temperatura Media (C)'].str.replace(',', '.').astype('float64')

file

,Data,Temperatura Media (C),Temperatura Minima (C),Temperatura Maxima (C),Precipitacao (mm),Final de Semana,Consumo de cerveja (litros),Dia da Semana
0,2015-01-01,27.30,"23,9","32,5",0,0.0,25.461,3.0
1,2015-01-02,27.02,"24,5","33,5",0,0.0,28.972,4.0
2,2015-01-03,24.82,"22,4","29,9",0,1.0,30.814,5.0
3,2015-01-04,23.98,"21,5","28,6","1,2",1.0,29.799,6.0
4,2015-01-05,23.82,21,"28,3",0,0.0,28.900,0.0
...,...,...,...,...,...,...,...,...
936,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
937,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
938,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
939,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
dataset = file.dropna()
dataset = pd.DataFrame(dataset, columns= ['Consumo de cerveja (litros)', 'Dia da Semana', 'Temperatura Media (C)'])

dataset['Consumo de cerveja (litros)'] = scipy.stats.zscore(dataset['Consumo de cerveja (litros)'])
# dataset['Consumo de cerveja (litros)'] = dataset.flatten()

dataset

,Consumo de cerveja (litros),Dia da Semana,Temperatura Media (C)
0,0.013574,3.0,27.30
1,0.812780,4.0,27.02
2,1.232072,5.0,24.82
3,1.001029,6.0,23.98
4,0.796390,0.0,23.82
...,...,...,...
360,1.571923,6.0,24.00
361,0.157891,0.0,22.64
362,-0.703913,1.0,21.68
363,-1.123205,2.0,21.38


In [18]:
X = dataset[['Dia da Semana','Temperatura Media (C)']]
y = dataset[['Consumo de cerveja (litros)']]

<h1> Outter cross validation

In [20]:
iteration = 1
for i in range(42,46):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=i)
    
    param_grid = [
    {'poly__degree': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
    'ridge__alpha':[0,1,2]}
      ]
    pipeline = Pipeline(steps=[('poly', PolynomialFeatures()), ('ridge', Ridge(normalize=True))])
    grid_search = GridSearchCV(pipeline, param_grid, cv=5,
                               scoring='r2',
                               refit=True)
    grid_search.fit(X_train, y_train)
    
    
    print("-------MELHORES PARÂMETROS-------")
    print(grid_search.best_params_)
    print("---------RESULTADOS---------")
    y_pred = grid_search.predict(X_test)
    print('Erro quadrado médio: %.2f'
      % mean_squared_error(y_test, y_pred))
    print('Coeficiente de determinação: %.2f'
      % r2_score(y_test, y_pred))
    print("-------------------------\n")
    
    iteration+=1

-------MELHORES PARÂMETROS-------
{'poly__degree': 3, 'ridge__alpha': 0}
---------RESULTADOS---------
Erro quadrado médio: 0.50
Coeficiente de determinação: 0.57
-------------------------

-------MELHORES PARÂMETROS-------
{'poly__degree': 4, 'ridge__alpha': 0}
---------RESULTADOS---------
Erro quadrado médio: 0.35
Coeficiente de determinação: 0.56
-------------------------

-------MELHORES PARÂMETROS-------
{'poly__degree': 4, 'ridge__alpha': 0}
---------RESULTADOS---------
Erro quadrado médio: 0.40
Coeficiente de determinação: 0.66
-------------------------

-------MELHORES PARÂMETROS-------
{'poly__degree': 4, 'ridge__alpha': 0}
---------RESULTADOS---------
Erro quadrado médio: 0.48
Coeficiente de determinação: 0.46
-------------------------



<h6>Para esse dataset, o melhor modelo tem o grau de polinômio igual à 4 e um fator de regularização igual à 0. 
Gerando um erro igual à 0.50 e um coeficiente de determinação próximo de 5.  